 <span style='background:gray;font-size:40px;'>  Setup  </span>

 <span style='background:gray;font-size:40px;'>  Importing Libraries  </span>

In [ ]:

from google.cloud import storage

import os
import random
import json
import gzip
import csv

import numpy as np
import pandas as pd

from urllib.request import urlopen
from tqdm import tqdm_notebook as tqdm
from collections import defaultdict
#from amazon.api import AMAZONAPI

 <span style='background:gray;font-size:40px;'>  Jupyter Options  </span>

In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

#pd.set_option('display.max_colwidth', None)

pd.set_option('display.max_colwidth', 100)

In [ ]:
# Already running in my instance since I checkboxed Anaconda
#%pip install more-itertools

***

# Import from Bucket - (Not needed since i uploaded directly onto persistent SSD)

In [ ]:
bucket_name = "finalproject-storage"

storage_client = storage.Client()
bucket = storage_client.get_bucket(bucket_name)

# # When you have your files in a subfolder of the bucket.
# my_prefix = "/" # the name of the subfolder
# blobs = bucket.list_blobs(prefix = my_prefix, delimiter = '/')

# for blob in blobs:
#     if(blob.name != my_prefix): # ignoring the subfolder itself 
#         file_name = blob.name.replace(my_prefix, "")
#         blob.download_to_filename(file_name) # download the file to the machine
#         df = pd.read_csv(file_name) # load the data
#         print(df)

# When you have your files on the first level of your bucket

blobs = bucket.list_blobs()


for blob in blobs:
    file_name = blob.name
    blob.download_to_filename(file_name) # download the file to the machine
    # df = pd.read_csv(file_name) # load the data
    # print(df)

In [ ]:
# Instantiate a Google Cloud Storage client and specify required bucket and file
storage_client = storage.Client()
bucket = storage_client.get_bucket('alouis9')
blob = bucket.blob('Electronics_5.json')

# Download the contents of the blob as a string and then parse it using json.loads() method
#json_data = blob.download_as_string().decode('utf-8')


In [ ]:
bucket.list_blobs()
blob
bucket

In [ ]:
for blob in blobs:
    file_name = blob.name
    blob.download_to_filename(file_name)

In [ ]:
data = pd.read_csv("gs://alouis9/Electronics_5.json")

***

<span style='background:gray;font-size:40px;'>  View Local Persistent SSD Tmp Folder where datafiles are stored  </span>

In [ ]:
import os
[s for s in os.listdir('/home/derek') if (s.endswith('.csv') | s.endswith('.gz'))]



# Nodes List w Image Link

In [ ]:
    path= '/home/derek/meta_Electronics.json.gz'
    def parse(path):
        g = gzip.open(path, 'rb')
        for l in g:
            yield json.loads(l)
    def getDF(whatev):
        i = 0
        df = {}
        for d in parse(whatev):
            df[i] = d
            i += 1
        return pd.DataFrame.from_dict(df, orient='index')

    dfnodes = getDF(path)
    dfnodes = dfnodes[['asin', 'title', 'imageURLHighRes']].explode('imageURLHighRes')
    dfnodes = dfnodes.groupby('asin').first().reset_index()

    dfnodes.to_csv(path.split('.')[0] + '.csv', index=False)

In [ ]:
dfnodes.head()
dfnodes[dfnodes.duplicated('asin')]
print("{}{:,}{}".format("There are ",dfnodes.asin.count()," unique asins w metadata \n"))


In [ ]:
dfnodes.head()
dfnodes[dfnodes.duplicated('asin')]
print("{}{:,}{}".format("There are ",dfnodes.asin.count()," unique asins w metadata \n"))



<span style='background:red;font-size:40px;'>  Import Ratings Only CSV Data  </span>
***

In [ ]:
path = '/home/derek/Electronics.csv'
header_list = ["item", "user", "rating", "timestamp"]
types_dict = {'item': 'string', 'user': 'string', 'rating': float, 'timestamp':'string'}
df_ratings = pd.read_csv(path, names=header_list, dtype=types_dict)
print("{}{:,}{}".format("There are ",len(df_ratings)," reviews in this dataset \n"))
df_ratings.info()
df_ratings = df_ratings.merge(dfnodes,how="left", left_on=['item'], right_on=['asin'])#[["item", "user", "rating", "timestamp", "title"]]
df_ratings.head()


<span style='background:green;font-size:40px;'> Metadata Exploration  </span>
***



<span style='background:green;font-size:40px;'> Rating Frequency Count </span>


In [ ]:
#df_ratings.style.format('{:,}')
ratingsFreq = df_ratings.groupby('rating').count().sort_values(['rating'],ascending=False)
ratingsFreq.style.format('{:,}')
print("{:,}{}{:,}{}".format(ratingsFreq.item.iloc[0]," out of ", ratingsFreq.item.sum(), " total reviews were rated 5 stars \n"))

In [ ]:
ratingspct = df_ratings.rating.value_counts(normalize=True)
print("{}{:,.1%}{}".format("",ratingspct.iloc[0]," of the reviews were rated 5 stars \n"))
ratingspct.to_frame().style.format('{:,.1%}')


<span style='background:green;font-size:40px;'> Average Rating Per Product </span>

In [ ]:
df_ratings.groupby(['title','item'])['rating'].mean().sort_values(ascending=False)

<span style='background:green;font-size:40px;'> Top 25 Most Reviewed Products </span>

In [ ]:
#num_reviews = df_ratings.groupby('item').count()
#num_reviews.sort_values(['rating'],ascending=False)
df_ratings.groupby(['title','item']).size().sort_values(ascending=False).head(25)#.agg('count')
#top100reviewers = rev_prolific.head(100).reset_index().copy()
#top100reviewers

<span style='background:green;font-size:40px;'> Top 10 Product with the most reviews per Rating </span>

In [ ]:
#df_ratings.groupby(['rating', 'item']).size().sort_values(ascending=False).nlargest(10)
ratagg = df_ratings.groupby(['title','rating', 'item']).agg({'user':'nunique'}).reset_index()
ratsort = ratagg.groupby(['rating']).apply(lambda x: x.sort_values(['user'],ascending = False)).reset_index(drop = True)
ratsort.groupby(['rating']).head(10).sort_values(['rating', 'user'],ascending=False)


<span style='background:green;font-size:40px;'> Top 25 Most Prolific Reviewers </span>

In [ ]:
# rev_prolific = df_ratings.groupby('user').count().sort_values(['rating'],ascending=False)
# top100reviewers = rev_prolific.head(100).reset_index().copy()
# top100reviewers

df_ratings.groupby('user').size().sort_values(ascending=False).head(25)

# Find The Product with most reviewer agreement for Each Rating

 <span style='background:red;font-size:40px;'> Load Category Product-Level Meta Data and create DF </span>

In [ ]:
path = '/home/derek/meta_Electronics.json.gz'
metadata = []
with gzip.open(path) as f:
    for l in f:
        metadata.append(json.loads(l.strip()))

In [ ]:
# convert list into pandas dataframe
df_metadata = pd.DataFrame.from_dict(metadata)
print(len(df_metadata))
df_metadata.columns
len(df_metadata[df_metadata.isna().any(axis=1)]) # Rows with NANS
df_metadata.info()

In [ ]:
type(metadata)
len(metadata)
type(df_metadata)
len(df_metadata)

In [ ]:
print("{}{:,}{}".format("There are ",len(metadata)," products w metadata \n"))

# first row of the list
print(metadata[0])

 <span style='background:red;font-size:40px;'> Load MASTER Product-Level Meta Data (all categories) and create DF </span>

In [ ]:
path = '/home/derek/title_alsobuy_asin_master.csv'
col_list = ["title", "asin"]
types_dict = {'title': 'string', 'alsobuy': 'string', 'asin': 'string'}
df_masterMETA = pd.read_csv(path, dtype=types_dict, usecols=col_list, skipinitialspace = True)

In [ ]:
print("{}{:,}{}".format("There are ",len(df_masterMETA)," items in this dataset \n"))
df_masterMETA.info()
#df_masterMETA = df_ratings.merge(dfnodes,how="left", left_on=['item'], right_on=['asin'])#[["item", "user", "rating", "timestamp", "title"]]
df_masterMETA.head()

 <span style='background:green;font-size:40px;'>Duplicate ASINS in Metadata Table</span>

In [ ]:
#df_metadata[df_metadata.duplicated('asin')].head()
len(df_metadata[df_metadata.duplicated('asin')])
len(df_alsobuys[df_alsobuys.duplicated('asin')])

<span style='background:green;font-size:40px;'> List of Duplicate asins in metadata</span>

In [ ]:


asin_dupes = df_metadata[df_metadata.duplicated('asin')].sort_values(by=['asin']).asin.tolist()

len(asin_dupes)

asin_dupes[0:10]
print("{}{:,}{}".format("There are ",len(asin_dupes)," asins with more than one row \n"))



<span style='background:green;font-size:40px;'>View Actual Duplicate Asins Side by Side in Metadata Table</span>

In [ ]:
df_metadata = df_alsobuys

df_asindupes = pd.concat(g for _, g in df_metadata.groupby('asin') if len(g) > 1)
df_asindupes.shape
print("{}{:,}{}{:,}{}".format("Since there are ",len(asin_dupes)," asins with more than one row, there are ", len(df_asindupes)," rows in total \n"))
df_asindupes.head(10)


# These are the rows with NAN in Details Column

In [ ]:
#df_metadata[df_metadata.isna().any(axis=1)]

In [ ]:
df_metadata[df_metadata.asin=='B01NAJ3KQB']

 <span style='background:red;font-size:40px;'> ***FILTER METADATA DF***</span>

In [ ]:
df_metadata[df_metadata.asin=='B00JHKSMG8']

***

 <span style='background:orange;font-size:40px;'>Co-purchases Search</span>

In [ ]:
no_copurch = len(df_metadata[(df_metadata.also_buy.str.len() == 0) & (df_metadata.also_view.str.len() == 0)])
both_copurch = len(df_metadata[(df_metadata.also_buy.str.len() > 0) & (df_metadata.also_view.str.len() > 0)])

print("{}{:,}{}".format("There are ",len(df_metadata)," asins in total in the metadata table \n"))

print("{}{:,}{}".format("There are ",no_copurch," asins with no also_buys or also_views \n"))

print("{}{:,}{}".format("There are ",len(df_metadata[df_metadata.also_buy.str.len() == 0])," asins with no also_buys \n"))

print("{}{:,}{}".format("There are ",len(df_metadata[df_metadata.also_view.str.len() == 0])," asins with no also_views \n"))

print("{}{:,}{}".format("There are ",len(df_metadata[df_metadata.also_buy.str.len() > 0])," asins with at least one also_buy co-purchase \n"))

print("{}{:,}{}".format("There are ",len(df_metadata[df_metadata.also_view.str.len() > 0])," asins with at least one also_views co-purchase \n"))

print("{}{:,}{}".format("There are ",both_copurch," asins with at least one of both co-purchase types \n"))

df_metadata[(df_metadata.also_buy.str.len() > 0) & (df_metadata.also_view.str.len() > 0)].head()
df_graph = df_metadata[(df_metadata.also_buy.str.len() > 0) & (df_metadata.also_view.str.len() > 0)]


<span style='background:orange;font-size:40px;'>Alternative Methodology to find Alsobuys exist?</span>

In [ ]:

df_metadata["also_buy"].apply(lambda x: 1 if len(x) == 0 else 0).sum()

<span style='background:orange;font-size:40px;'>Build Subset df to eliminate asins with no alsobuys</span>

In [ ]:
df_alsobuys = df_metadata[df_metadata.also_buy.str.len() > 0]

In [ ]:
len(df_alsobuys)
print("{}{:,}{}".format("There are ",len(df_metadata[df_metadata.also_buy.str.len() > 0])," asins with at least one also_buy co-purchase \n"))

<span style='background:orange;font-size:70px;'>At this point there is no reason to keep working with the full category metadata set, so we can eliminate any rows with ZERO also-buys</span>

<div class="alert alert-block alert-info">
    <b><p style="font-size:30px">Turn this flag on to replace full metadata df with subset limited to asins with alsobuys</b></p>
</div>

In [ ]:
df_metadata = df_alsobuys
df_metadata['asin'].str.strip()
len(df_metadata)
len(df_metadata)==len(df_alsobuys)

In [ ]:
df_metadata.head()

***

<div class="alert alert-block alert-info">
    <b><p style="font-size:30px">There are multiple ways to build an edgelist</b></p>
</div>

# Create Dictionary Form of Co-Purchases by Asin

In [ ]:
# There are two different dictionary structures


## Dictionary where the keys are ['asin','title', 'also_buy'] and the values are a dictionary with indexed, attrb

In [ ]:
alsodict = df_metadata[['asin','title', 'also_buy']].to_dict()


In [ ]:
# [{0: '0011300000',
#   1: '0043396828',
#   2: '0060009810',

#list(alsodict.values())[:1]

# [{0: 'Genuine Geovision 1 Channel 3rd Party NVR IP Software with USB Dongle Onvif PSIA',
#   1: 'Books "Handbook of Astronomical Image Processing" with CD ROM, 2nd Edition, Hardcover Book by Berry &amp; Burnell',
#   2: 'One Hot Summer',
#   3: 'Hurray for Hattie Rabbit: Story and pictures (An Early I can read book)',
#   4: 'sex.lies.murder.fame.: A Novel',
#   5: 'College Physics',

In [ ]:
## Dict where the ASIN is the key and 

In [ ]:
#metadf[['asin','title', 'also_buy']].dropindex()
meta_df = df_metadata[['asin','title', 'also_buy']].set_index('asin')
#meta_dffinal = df_graph[['asin','title', 'also_buy']].set_index('asin')
meta_dict = meta_df.to_dict()
#meta_dictfinal = meta_dffinal.to_dict()


## Dict where the keys are ['title', 'also_buy'] and the values are [asin, attr]

In [ ]:
meta_df.head()
#list(meta_dict.items())[0:1]
meta_dict.keys()

In [ ]:
# [('title',
#   {'0011300000': 'Genuine Geovision 1 Channel 3rd Party NVR IP Software with USB Dongle Onvif PSIA',
#    '0043396828': 'Books "Handbook of Astronomical Image Processing" with CD ROM, 2nd Edition, Hardcover Book by Berry &amp; Burnell',
#    '0060009810': 'One Hot Summer',
#    '0060219602': 'Hurray for Hattie Rabbit: Story and pictures (An Early I can read book)',

In [ ]:
alsodict.keys()
meta_dict.keys()
#meta_dictfinal.keys()
type(meta_dict['also_buy'])
#len(meta_dictfinal['also_view'])
#meta_df
#meta_dict['also_buy']

# ASINS Sorted/Ranked by Number of Also_Buy

In [ ]:
def sort_by_values_len(dict):
    dict_len= {key: len(value) for key, value in dict.items()}
    import operator
    sorted_key_list = sorted(dict_len.items(), key=operator.itemgetter(1), reverse=True)
    sorted_dict = [{item[0]: dict[item [0]]} for item in sorted_key_list]
    return sorted_dict

# Sorted Dictionary where key is asin and value is alsobuys list

In [ ]:
alsobuys_rnk = sort_by_values_len(meta_dict['also_buy'])

In [ ]:
#sorted(meta_dict['also_buy'].items(), key=lambda e: len(e))
type(alsobuys_rnk)
len(alsobuys_rnk)

type(alsobuys_rnk[0])
len(alsobuys_rnk[0:2])

In [ ]:

# This Dictionary is 2996 rows less which is correct because it has removed the duplicate rows and combined the values

## Verification that the dictionary is ranked and the 100,776th row has no alsobuys (therefore doesn't exist)

In [ ]:
# You can see how the dictionary is ranked by the number of alsobuys
alsobuys_rnk[1].values()
alsobuys_rnk[100775].values()
alsobuys_rnk[100776].values()


In [ ]:

#gph = nx.from_dict_of_dicts(alsobuys_rnk)

<div class="alert alert-block alert-info">
    <b><p style="font-size:30px">Create an Edge Set Manually</b></p>
</div>

In [ ]:
#alsobuys_rnk[0]['0545105668']

In [ ]:

edgeset = set()
for i in range(len(alsobuys_rnk)):
    for asin in alsobuys_rnk[i]: 
        for alsobuy in alsobuys_rnk[i][asin]: # For Every Alsobuy in an alsobuy list
            edgeset.add(tuple(sorted((asin,alsobuy)))) # changes order of tuples 
        


<div class="alert alert-block alert-info">
    <b><p style="font-size:30px">Create an Edge LIST</b></p>
</div>

In [ ]:
edgelist = []

for i in range(len(alsobuys_rnk)):
    for asin in alsobuys_rnk[i]:
        for alsobuy in alsobuys_rnk[i][asin]:
            edgelist.append((asin, alsobuy))


<div class="alert alert-block alert-info">
    <b><p style="font-size:30px">Note that they are of Different Lengths (probably list has duplicates)</b></p>
</div>

In [ ]:
type(edgeset)
len(edgeset)
type(edgelist)
len(edgelist)
# The edgelist has 61,000 more than the set

<div class="alert alert-block alert-info">
    <b><p style="font-size:30px">When Creating a Set from the List this way, the number of edges is the same</b></p>
</div>


In [ ]:
res = set(map(lambda x: tuple(sorted(x)), edgelist))

In [ ]:
len(res) #Interesting same length as set above
res==edgeset

<div class="alert alert-block alert-info">
    <b><p style="font-size:30px">The above is proof that creating the set of edges is accurate and the list is not</b></p>
</div>


In [ ]:
# Same number of asins
len(meta_dict['also_buy'].keys())
len(alsobuys_rnk)

***

<div class="alert alert-block alert-info">
    <b><p style="font-size:30px">Annabels formula was built using the raw metadata file, not the df, so to make is less expensive I am lobbing off all rows we already know have no alsobuys<br>
<br>I already turned on the flag above to make this conversion so I am just reverting the df back to a list
<br>Create SUBSET METADATA TO LOWER COMPUTATION COST OF ANNABELS FORMULA</b></p>
</div>

In [ ]:
metadatab = df_metadata.to_dict('records')

In [ ]:
type(metadatab)
len(metadatab)

<div class="alert alert-block alert-info">
    <b><p style="font-size:30px">Annabels Edgelist Function</b></p>
</div>


In [ ]:
dict1 = {}

for item1 in metadatab: 
    for item2 in item1['also_buy']:
        if (item1['asin'],item2) and (item2, item1['asin']) not in dict1: 
            dict1[(item1['asin'],item2)] = 1
        elif (item1['asin'], item2) in dict1: 
            dict1[(item1['asin'],item2)] += 1
        elif (item2, item1['asin']) in dict1:
            dict1[(item2,item1['asin'])] += 1
    list_of_pairs = [(item1['also_buy'][p1], item1['also_buy'][p2]) for p1 in range(len(item1['also_buy'])) for p2 in range(p1+1,len(item1['also_buy']))]
    # for every alsobuy
    for (item3,item4) in list_of_pairs: 
        if(item3, item4) and (item4, item3) not in dict1: 
             dict1[(item3,item4)] = 1
        elif (item3,item4) in dict1: 
            dict1[(item3,item4)] += 1
        elif (item4,item3) in dict1: 
            dict1[(item4,item3)] += 1

In [ ]:
list_of_pairs = [(metadatab[3]['also_buy'][p1], metadatab[3]['also_buy'][p2]) for p1 in range(len(metadatab[3]['also_buy'])) for p2 in range(p1+1,len(metadatab[3]['also_buy']))]

In [ ]:
metadatab[3]['also_buy']
len(list_of_pairs)
list_of_pairs

# This is not correct, because you have no idea if these list of pairs are bought together, you only know that each one was bought with item1[asin]



<div class="alert alert-block alert-info">
    <b><p style="font-size:30px">Correct Edgelist Approach w Weights</b></p>
</div>


In [ ]:
dict2 = {}

for item1 in metadatab: 
    for item2 in item1['also_buy']:
        if (item1['asin'],item2) and (item2, item1['asin']) not in dict2: 
            dict2[(item1['asin'],item2)] = 1
        elif (item1['asin'], item2) in dict2: 
            dict2[(item1['asin'],item2)] += 1
        elif (item2, item1['asin']) in dict2:
            dict2[(item2,item1['asin'])] += 1
    list_of_pairs = [(item1['also_buy'][p1], item1['also_buy'][p2]) for p1 in range(len(item1['also_buy'])) for p2 in range(p1+1,len(item1['also_buy']))]
    for (item3,item4) in list_of_pairs: 
#          if(item3, item4) and (item4, item3) not in dict1: 
#              dict1[(item3,item4)] = 1
        if (item3,item4) in dict2: 
            dict2[(item3,item4)] += 1
            break
        elif (item4,item3) in dict2: 
            dict2[(item4,item3)] += 1

In [ ]:
# In this formula we take all the pairs of items in a single also bought list and then check to see if they are already in the edgelist. If they are we increase their weight by 1
# This is correct because we are not adding any new edges we are just giving freqently seen alsobought added focus/weight

<div class="alert alert-block alert-info">
    <b><p style="font-size:30px">Basic Unweighted Edgeweight Approach, identical to networkx and my approach above</b></p>
</div>


In [ ]:
dict3 = {}

for item1 in metadatab: 
    for item2 in item1['also_buy']:
        if (item1['asin'],item2) and (item2, item1['asin']) not in dict3: 
            dict3[(item1['asin'],item2)] = 1
        elif (item1['asin'], item2) in dict3: 
            dict3[(item1['asin'],item2)] += 1
        elif (item2, item1['asin']) in dict3:
            dict3[(item2,item1['asin'])] += 1

In [ ]:
#len(dict1)
len(dict2)
len(dict3)

#max(dict1.values())
max(dict2.values())
max(dict3.values())



import itertools
#dict(itertools.islice(dict2.items(), 10))
#sorted(dict2.items(), key=lambda x: x[1],reverse=True)

# 29319645
# 1818275
# 1818275
# 271
# 109
# 3

In [ ]:
new = pd.DataFrame(edgeset)
neww = pd.DataFrame(dict2.keys())

In [ ]:
new["lookupkey"] = new[0] + new[1]
neww["lookupkey"] = neww[0] + neww[1]

In [ ]:
new.head()
#new.info()
neww.head()
#neww.info()
new.sort_values(by='lookupkey').head()
neww.sort_values(by='lookupkey').head()

In [ ]:
len(pd.DataFrame(dict2.keys()))
len(pd.DataFrame(edgeset))
d2l = pd.DataFrame(dict2.keys()).tolist()
esl = pd.DataFrame(edgeset).tolist()

In [ ]:
#pd.DataFrame.from_dict(dict2, index=[0]).head()


ddf = pd.DataFrame(dict2.items())#.astype(str)
ddf.head()

In [ ]:
ddfcopy = ddf.sort_values(by=1, ascending=False).reset_index()

In [ ]:
ddfcopy.head()
srctgt = pd.DataFrame(ddfcopy[0].tolist())

srctgt["lookupkey"] = srctgt[0] + srctgt[1]
dfwkey = srctgt.astype(str).copy()

#dfwkey.merge(ddfcopy, how=[''], 

In [ ]:
ddfcopy[ddfcopy[1]==101][0]

In [ ]:
len(ddfcopy)
len(dfwkey)
dfwkey.head()
dfwkey.info()

In [ ]:
#dfapp = ddfcopy.append(dfwkey['lookupkey'],sort=False)

In [ ]:
#dfapp.head()

df_weights = pd.concat([ddfcopy, dfwkey['lookupkey']], axis=1)#.astype(str)
df_weights.columns = ['idx', 'edge', 'edge_wt', 'lookup']
df_weights.head()
df_weights.info()
len(df_weights)
df_weights = df_weights.apply(lambda x: x.str.strip() if x.dtype == "object" else x)

In [ ]:
df_weights[df_weights.lookup=='B0000BVYT3B015PRO512']

In [ ]:
(B000FBK3QKB014JV57KS)

In [ ]:
dftuples = pd.DataFrame(ddf[0].tolist())
type(dftuples)
dftuples.head()



<div class="alert alert-block alert-info">
    <b><p style="font-size:30px"> Create Edge set using the networkx built-in function</b></p>
</div>

In [ ]:
import networkx as nx
g = nx.from_dict_of_lists(meta_dict['also_buy'])


In [ ]:
len(meta_dict['also_buy'])

# It has the same number of edges as our manual edgeliset but different number of nodes?

In [ ]:
g.size()
type(g)
len(g)
len(g.nodes())
# This matches the manual method above

In [ ]:

import itertools

In [ ]:
dir(itertools)

In [ ]:
type(edgeset)
set(itertools.islice(edgeset, 10))

In [ ]:
asinlist = list(edgeset)
len(asinlist)
asinlist[0:5]
ab_asins = pd.DataFrame(asinlist, columns=['source','target'])
len(ab_asins)
ab_asins.head()
type(edgeset)

#df_ratings = df_ratings.merge(dfnodes,how="left", left_on=['item'], right_on=['asin'])

In [ ]:
## These needs to be an innerjoin followed by a left join with drop na, because if either source or node is missing metadata we need to drop

In [ ]:
dfwkey.columns = ['source', 'target', 'lookup']
len(dfwkey)
len(ab_asins)
dfwkey.head()
ab_asins.head()


In [ ]:
dfmerge1 = dfwkey.merge(df_masterMETA.rename(columns={'title': 'src_title'}), how='inner', left_on=['target'], right_on=['asin'])


In [ ]:
len(dfmerge1)
dfmerge1.head()

In [ ]:
dffin = dfmerge1.merge(df_masterMETA.rename(columns={'title': 'target_title'}), how='left', left_on=['source'], right_on=['asin']).dropna()

In [ ]:
len(dffin)
dffin[['source', 'target', 'src_title', 'target_title']].head()

In [ ]:
dffin["lookup"] = dffin["source"] + dffin["target"]

In [ ]:
dfff = dffin.copy()
dfff = dfff.apply(lambda x: x.str.strip() if x.dtype == "object" else x)
dfff.head()
len(dfff)

In [ ]:
df_last = dfff.merge(df_weights, how='left', on=['lookup'])[['source','target','src_title','target_title', 'lookup', 'edge_wt']].sort_values(by='edge_wt', ascending=False).reset_index(drop=True)

In [ ]:
df_last.head()

In [ ]:
df_last[df_last.edge_wt.isnull()]

In [ ]:
df_last.info()

In [ ]:
# compression_opts = dict(method='zip',
#                         archive_name='FINALELECTRONICS.csv')  
df_last.to_csv('FINALELECTRONICS.csv', index=False)  

In [ ]:
os.listdir()

In [ ]:
#ab_asins.columns
alsobuyseries = ab_asins.iloc[:, 0]
len(alsobuyseries)

In [ ]:
#ab_asins.melt('topic', value_name='key').drop('variable', 1)
df_nodeslist = pd.melt(ab_asins, value_vars=['source', 'target'], value_name='asin')
len(df_nodeslist)
df_nodesfinal = df_nodeslist.drop_duplicates(subset=['asin'])

In [ ]:
len(df_nodesfinal)
df_nodesfinal.head()

In [ ]:
df_edgenames = df_nodesfinal.merge(df_masterMETA,how="inner", left_on=['asin'], right_on=['asin'])#[['variable','asin','title']]

In [ ]:
len(df_edgenames)
df_edgenames.head()

In [ ]:
df_edgenames.pivot(columns='variable')#['asin','title']

# This show the number of unique nodes and matches the built in networkx

In [ ]:
len(pd.unique(ab_asins[['source', 'target']].values.ravel('K')))

In [ ]:
len(df_nodeslist.asin.unique())

In [ ]:
dfnodes.set_index('asin').merge(alsobuyseries.rename('source'), left_index=True, right_index=True)

In [ ]:
dfnodes.set_index('asin').head()

In [ ]:
also_buys_diff_cat = ab_asins.merge(dfnodes,how="left", left_on=['target'], right_on=['asin'])[['source','target','asin','title']]

In [ ]:
also_buys_diff_cat

# Number of Also_Buys from an Outside Cateogry

In [ ]:
#also_buys_diff_cat[[also_buys_diff_cat.asin.isnan()]].head()

also_buys_diff_cat[also_buys_diff_cat['asin'].isnull()].head(5)

print("{}{:,}{}".format("There are ",len(also_buys_diff_cat[also_buys_diff_cat['asin'].isnull()])," asins in our edgelist from an outside category \n"))

In [ ]:
also_buys_diff_cat[also_buys_diff_cat.source=='B01NAJ3KQB']


<div class="alert alert-block alert-info">
    <b><p style="font-size:30px">Join in New Nodes</b></p>
</div>

# Merge New MetaData into edgelist

In [ ]:
df_edge1 = ab_asins.merge(df_masterMETA.rename(columns={'title': 'target_title'}),how="left", left_on=['target'], right_on=['asin'])



In [ ]:
len(df_edge1)
df_edge1.head()
ab_asins.head()

In [ ]:
df_edge2 = ab_asins.merge(df_masterMETA.rename(columns={'title': 'src_title'}),how="left", left_on=['source'], right_on=['asin'])

In [ ]:
len(df_edge2)
df_edge2.head()



In [ ]:
dfwtf = df_edge2.merge(df_edge1,how="left", on='target')

In [ ]:
len(dfwtf)
dfwtf.head()

In [ ]:
#df_edge3 = ab_asins.merge(df_masterMETA,how="left", left_on=['source','target'], right_on=['asin'])
dfconcat = pd.concat([df_edge1, df_edge2])
#df = pd.concat([ab_asins.merge(df_masterMETA,how="inner", left_on=['target'], right_on=['asin']), ab_asins.merge(df_masterMETA,how="inner", left_on=['source'], right_on=['asin'])])

In [ ]:
df_hope = dfconcat[dfconcat.notna()]

In [ ]:
len(df_hope)
df_hope.head()

In [ ]:
#df.head()
df_hope = df[df['asin'].notna()]
len(df_hope)

In [ ]:
df_hope.head(100)

In [ ]:
df_hope[df_hope.asin=='B000CDHQK2']

In [ ]:
df_edge1.head()
df_edge2.head()

 # I belive this means that it is adding nodes from the alsobuys that don't exist in the metadata table for the category

In [ ]:
dict1.values

In [ ]:

list(g.edges(data=True))[0:1000]

In [ ]:
len(g.edges())
len(meta_dict['also_buy'])
type(nx.to_edgelist(g)[0:2])

In [ ]:


# for d in alsobuys_rnk:
#     g = nx.from_dict_of_lists(alsobuys_rnk[0])
# g.edges()
# type(g)

import networkx as nx
graphs = []
for i in range(len(alsobuys_rnk)):
    g = nx.from_dict_of_lists(alsobuys_rnk[i])
    graphs.append(g)
final_graph = nx.compose_all(graphs)

In [ ]:
final_graph = nx.compose_all(graphs)

In [ ]:
len(final_graph.edges())
len(graphs)

In [ ]:
type(final_graph)

In [ ]:
edges = []
nodes = []
for dict in alsobuys_rnk:
    for i in dict:
        nodes.append(i)

In [ ]:
len(nodes)


In [ ]:
range(len(alsobuys_rnk[0]))

In [ ]:
type(alsobuys_rnk)
len(alsobuys_rnk)

In [ ]:
for asin in alsobuys_rnk[1]: print(asin)

In [ ]:
for dict in alsobuys_rnk: print(node)

In [ ]:
generate_edges(alsobuys_rnk)

In [ ]:
edgesset = generate_edgesset(alsobuys_rnk)

In [ ]:
len(edgeslist)
edgeslist

In [ ]:
edgeslistlist = generate_edgeslist(alsobuys_rnk)

In [ ]:
len(edgeslistlist)
edgeslistlist[0:50]

# Function to Create CSV Edgelist of 1 ASIN

In [ ]:
# note: If you use 'b' for the mode, you will get a TypeError
# under Python3. You can just use 'w' for Python 3

with open('edgelist.csv','w') as out:
    csv_out=csv.writer(out)
    csv_out.writerow(['name','num'])
    for row in edgeslist:
        csv_out.writerow(row)

    # You can also do csv_out.writerows(data) instead of the for loop

In [ ]:
#alsodict.keys()
meta_df.keys()

In [ ]:
list(alsodict)[:3]

In [ ]:
#alsodict

In [ ]:
import more_itertools
#more_itertools.take(1, alsodict.items())
#{k: alsodict[k] for k in list(alsodict)[:3]}
more_itertools.take(3, meta_df.items())

In [ ]:
len(alsodict.keys())

In [ ]:
len(alsodict['asin'])

In [ ]:
len(alsodict['title'])

In [ ]:
len(alsodict['also_buy'])

In [ ]:
alsobuy_v1 = df["also_buy"].apply(pd.Series)

In [ ]:
alsobuy_v1

In [ ]:
edgelist = '/edgelist.csv'
#header_list = ["asin", "also_buy"]
df_edgelist = pd.read_csv(edgelist)
df_edgelist.head()
len(df_edgelist)

In [ ]:
nodeslist=set(df_edgelist.name)

In [ ]:
len(nodeslist)

In [ ]:
with open('edgelist.csv','w') as out:
    csv_out=csv.writer(out)
    csv_out.writerow(['name','num'])
    for row in edgeslist:
        csv_out.writerow(row)


In [ ]:
df_metadata[df_metadata.also_buy.str.len() > 0]

In [ ]:
#Creating Network Graph
Gr = nx.Graph()
Gr.add_edges_from(edge_tuples)

In [ ]:
from networkx.algorithms.community.centrality import girvan_newman
#Implementing Girvan_Newman 
girvan_groups = []
girvan_communities = girvan_newman(g)
for com in next(girvan_communities): 
    girvan_groups.append(list(com))

In [ ]:
type(girvan_groups)
girvan_groups
girvan_communities

In [ ]:
color_map = []
for node in g:
    if node in girvan_groups[0]:
        color_map.append('blue')
    else: 
        color_map.append('green')  
nx.draw(g)
plt.show()

In [ ]:
G_small_world = nx.watts_strogatz_graph(n=100, k = 5, p = 0.4) 
#n=number of nodes, k=number of nearby links, p = probability of rewiring a link to a far away node

In [ ]:
%matplotlib q
nx.draw_random(g)

In [ ]:
#pos=nx.spring_layout(g)
#nx.draw(g,pos,node_color='#A0CBE2',edge_color='#BB0000',width=2,edge_cmap=plt.cm.Blues,with_labels=False)

In [ ]:
!pip install graph_tool

In [ ]:
import graph_tool.all as gt

In [ ]:
from graph_tool.all import *


In [ ]:
dir(gt.draw)

In [ ]:
import networkx as nx
import graph_tool as gt
import cairo

In [ ]:
#!apt-get install libcairo2-dev
#!pip install pycairo 
!pip install pycairo==1.11.1
#!pip uninstall pycairo --y

In [ ]:




def get_prop_type(value, key=None):
    """
    Performs typing and value conversion for the graph_tool PropertyMap class.
    If a key is provided, it also ensures the key is in a format that can be
    used with the PropertyMap. Returns a tuple, (type name, value, key)
    """
    if isinstance(key, unicode):
        # Encode the key as ASCII
        key = key.encode('ascii', errors='replace')

    # Deal with the value
    if isinstance(value, bool):
        tname = 'bool'

    elif isinstance(value, int):
        tname = 'float'
        value = float(value)

    elif isinstance(value, float):
        tname = 'float'

    elif isinstance(value, unicode):
        tname = 'string'
        value = value.encode('ascii', errors='replace')

    elif isinstance(value, dict):
        tname = 'object'

    else:
        tname = 'string'
        value = str(value)

    return tname, value, key


def nx2gt(nxG):
    """
    Converts a networkx graph to a graph-tool graph.
    """
    # Phase 0: Create a directed or undirected graph-tool Graph
    gtG = gt.Graph(directed=nxG.is_directed())

    # Add the Graph properties as "internal properties"
    for key, value in nxG.graph.items():
        # Convert the value and key into a type for graph-tool
        tname, value, key = get_prop_type(value, key)

        prop = gtG.new_graph_property(tname) # Create the PropertyMap
        gtG.graph_properties[key] = prop     # Set the PropertyMap
        gtG.graph_properties[key] = value    # Set the actual value

    # Phase 1: Add the vertex and edge property maps
    # Go through all nodes and edges and add seen properties
    # Add the node properties first
    nprops = set() # cache keys to only add properties once
    for node, data in nxG.nodes(data=True):

        # Go through all the properties if not seen and add them.
        for key, val in data.items():
            if key in nprops: continue # Skip properties already added

            # Convert the value and key into a type for graph-tool
            tname, _, key  = get_prop_type(val, key)

            prop = gtG.new_vertex_property(tname) # Create the PropertyMap
            gtG.vertex_properties[key] = prop     # Set the PropertyMap

            # Add the key to the already seen properties
            nprops.add(key)

    # Also add the node id: in NetworkX a node can be any hashable type, but
    # in graph-tool node are defined as indices. So we capture any strings
    # in a special PropertyMap called 'id' -- modify as needed!
    gtG.vertex_properties['id'] = gtG.new_vertex_property('string')

    # Add the edge properties second
    eprops = set() # cache keys to only add properties once
    for src, dst, data in nxG.edges(data=True):

        # Go through all the edge properties if not seen and add them.
        for key, val in data.items():
            if key in eprops: continue # Skip properties already added

            # Convert the value and key into a type for graph-tool
            tname, _, key = get_prop_type(val, key)

            prop = gtG.new_edge_property(tname) # Create the PropertyMap
            gtG.edge_properties[key] = prop     # Set the PropertyMap

            # Add the key to the already seen properties
            eprops.add(key)

    # Phase 2: Actually add all the nodes and vertices with their properties
    # Add the nodes
    vertices = {} # vertex mapping for tracking edges later
    for node, data in nxG.nodes(data=True):

        # Create the vertex and annotate for our edges later
        v = gtG.add_vertex()
        vertices[node] = v

        # Set the vertex properties, not forgetting the id property
        data['id'] = str(node)
        for key, value in data.items():
            gtG.vp[key][v] = value # vp is short for vertex_properties

    # Add the edges
    for src, dst, data in nxG.edges(data=True):

        # Look up the vertex structs from our vertices mapping and add edge.
        e = gtG.add_edge(vertices[src], vertices[dst])

        # Add the edge properties
        for key, value in data.items():
            gtG.ep[key][e] = value # ep is short for edge_properties

    # Done, finally!
    return gtG

In [ ]:
gtG = nx2gt(g)

In [ ]:
 gtG.list_properties()

In [ ]:
gtG

In [ ]:
g2 = gt.Graph(gtG)      

In [ ]:
g2

In [ ]:
pos = fruchterman_reingold_layout(g2, n_iter=1000)
graphviz_draw(g2, pos=pos, output="graph-draw-fr.pdf")

In [ ]:
graph_draw

In [ ]:
pos = gt.draw.arf_layout(gtG, max_iter=0)
gt.draw.graphviz_draw(gtG, pos=pos, output="graph-draw-arf.pdf")

In [ ]:
gt.draw.graphviz_draw(gtG, pos=pos, output="graph-draw-arf.png")

In [ ]:
print("fuck")

In [ ]:
import sys
print(sys.executable)
print(sys.path)

In [ ]:
import os
import sys
sys.path.insert(0, os.path.abspath('/usr/lib/python3/dist-packages/graph_tool'))

In [ ]:
g